In [79]:
!pip install streamlit pyngrok langchain faiss-cpu sentence-transformers PyPDF2 transformers


In [80]:
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "32SriXYHFbQrT4A5wCPVxgWbv5p_6YyEXLVRuRknXtvY2283P"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


In [81]:
%%writefile app.py
import os
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Streamlit Page Setup
st.set_page_config(page_title="Course Material Q&A Assistant", layout="wide")
st.title("📚 Course Material Q&A Assistant")
st.markdown("Upload your PDFs and ask questions!")

# Directory to store PDFs
UPLOAD_DIR = "uploaded_pdfs"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# PDF Upload
uploaded_files = st.file_uploader("Upload PDF files", type="pdf", accept_multiple_files=True)

if uploaded_files:
    all_texts = []
    for uploaded_file in uploaded_files:
        file_path = os.path.join(UPLOAD_DIR, uploaded_file.name)
        with open(file_path, "wb") as f:
            f.write(uploaded_file.getbuffer())
        pdf = PdfReader(file_path)
        pdf_text = ""
        for page in pdf.pages:
            pdf_text += page.extract_text() + "\n"
        all_texts.append(pdf_text)
    st.success(f"✅ {len(uploaded_files)} PDFs processed successfully!")

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
    docs = []
    for text in all_texts:
        docs.extend(text_splitter.split_text(text))
    st.info(f"Text split into {len(docs)} chunks.")

    # FAISS Vector Store
    VECTORSTORE_PATH = "faiss_index"
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    if os.path.exists(VECTORSTORE_PATH):
        vectorstore = FAISS.load_local(
            VECTORSTORE_PATH,
            embeddings,
            allow_dangerous_deserialization=True  # <-- FIX added
        )
        st.success("✅ Loaded existing FAISS vector store.")
    else:
        vectorstore = FAISS.from_texts(docs, embeddings)
        vectorstore.save_local(VECTORSTORE_PATH)
        st.success("✅ Created new FAISS vector store.")

    # Local LLM using GPT-Neo
    llm_pipeline = pipeline(
        "text-generation",
        model="EleutherAI/gpt-neo-125M",
        max_length=1000,
        do_sample=True,
        temperature=0.7
    )
    llm = HuggingFacePipeline(pipeline=llm_pipeline)

    # Retrieval QA
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        chain_type="stuff"
    )

    # Chat Interface
    question = st.text_input("Ask a question about your PDFs:")
    if question:
        with st.spinner("Generating answer..."):
            answer = qa.run(question)
        st.markdown(f"**Answer:** {answer}")


Overwriting app.py


In [82]:
import os
import time
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Open new Ngrok tunnel
public_url = ngrok.connect(8501)
print("🚀 Your public Streamlit URL:", public_url)

# Run Streamlit app in the background
os.system("streamlit run app.py &")

# Wait a few seconds to ensure Streamlit starts
time.sleep(5)
print("Streamlit should now be running. Open the Ngrok URL above in your browser.")


🚀 Your public Streamlit URL: NgrokTunnel: "https://8e3ca8975339.ngrok-free.app" -> "http://localhost:8501"
Streamlit should now be running. Open the Ngrok URL above in your browser.
